In [1]:
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
import re

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

/home/husein/3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/husein/3.9/lib/python3.9/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/3.9/lib/python3.9/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [2]:
openai.api_type = 'azure'
openai.api_base = 'https://nous.openai.azure.com/'
openai.api_version = '2023-07-01-preview'
openai.api_key = ''

In [3]:
engine = 'nous'

In [4]:
def predict(text):
    try:
        messages = [
            {"role": "user", "content": text},
        ]
        response = openai.ChatCompletion.create(
            engine=engine,
            messages=messages,
            temperature=0.7,
            max_tokens=2048,
            top_p=0.95,
        )
        return response.choices[0]['message']['content']
    except Exception as e:
        print(e)
        return None

In [5]:
data = []
with open('paragraph-wiki-v2.jsonl') as fopen:
    for l in fopen:
        data.append(json.loads(l))
        
len(data)

143828

In [6]:
data[1]

{'paragraph': 'yang terletak lebih selatan yakni kawasan maritim Asia Tenggara Secara umumnya, garis perkembangan bahasa Melayu terbahagi kepada tiga fasa utama iaitu bahasa Melayu Kuno, bahasa Melayu Klasik dan bahasa Melayu Moden, dan terdapat fasa peralihan di antara fasa-fasa utama tersebut. Sesetengah pakar bahasa menganggap bahasa Melayu Kuno ialah bahasa leluhur bahasa Melayu Klasik, tetapi sesetengah ahli linguistik yang lain pula berpendapat bahawa hubungan leluhur antara dua fasa bahasa ini susah untuk ditentusahkan berikutan keganjilan dari segi morfologi dan sintaksis antara dua fasa ini, serta keserentakan penggunaan kedua-dua fasa bahasa ini pada abad ke-14. Bahasa Melayu Kuno dipengaruhi oleh bahasa Sanskrit yang merupakan bahasa klasik di India. Peminjaman kata-kata Sanskrit boleh didapati dalam perbendaharaan bahasa Melayu Kuno. Batu bersurat terawal dalam bahasa Melayu Kuno dijumpai di Sumatra, Indonesia, dan ditulis dalam huruf-huruf Grantha bervariasi Pallava dan ta

In [7]:
# !rm -rf wikipedia-question
!mkdir wikipedia-question

mkdir: cannot create directory ‘wikipedia-question’: File exists


In [8]:
def generate(filename, l):
    if os.path.exists(filename):
        return
    
    s = f"""
{l['paragraph']}

generate a question related to the context
"""
    s = s.strip()
    r = predict(s)
    l['question'] = r

    with open(filename, 'w') as fopen:
        json.dump(l, fopen)

In [9]:
generate('wikipedia-question/0.jsonl', data[0])

In [ ]:
from tqdm import tqdm

max_worker = 2
for i in tqdm(range(0, len(data), max_worker)):
    b = data[i: i + max_worker]
    filenames = [(os.path.join('wikipedia-question', f'{i + k}.json'), l) for k, l in enumerate(b)]
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(generate, *f): f for f in filenames}

        for future in as_completed(futures):
            future.result()

 34%|███████████▋                      | 24757/71914 [05:58<19:10:27,  1.46s/it]

'content'


 34%|███████████▋                      | 24765/71914 [06:09<18:01:05,  1.38s/it]

'content'


 35%|███████████▉                      | 25318/71914 [19:26<19:19:23,  1.49s/it]

...

 66%|█████████████████████           | 47222/71914 [9:16:30<10:01:34,  1.46s/it]

The response was filtered due to the prompt triggering Azure OpenAI’s content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


 66%|█████████████████████▎          | 47783/71914 [9:29:35<10:29:07,  1.56s/it]

The response was filtered due to the prompt triggering Azure OpenAI’s content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


 66%|█████████████████████▉           | 47793/71914 [9:29:49<9:46:10,  1.46s/it]